In [1]:

import requests
import pandas as pd
import time
from datetime import datetime, timedelta
import numpy as np
import os

In [2]:
def generate_date_range(start_date, end_date):
    start_date = datetime.strptime(start_date, '%Y-%m-%d')
    end_date = datetime.strptime(end_date, '%Y-%m-%d')

    date_list = []
    current_date = start_date
    while current_date <= end_date:
        date_list.append(current_date.strftime("%d%m%Y"))
        current_date += timedelta(days=1)

    return date_list


date_range = generate_date_range('2024-02-06', '2024-03-08')

ts=[]
for i in range(len(date_range)):
    date = date_range[i]
    link = 'https://promo.betfair.com/betfairsp/prices/dwbfgreyhoundwin'+date+'.csv'
    t1 = time.time()
    try:
        df1 = pd.read_csv(link)
        df1.columns = df1.columns.str.upper()
    except Exception as e:
        print(e)
    
    if(date==date_range[0]):
        df = df1.copy()
    else:
        df = pd.concat([df,df1])
    
    t2 = time.time()
    
    t=round(t2-t1,3)
    print(i,date,'time taken: ',t)
    ts.append(t)
print('avg:',round(sum(ts)/len(ts),3))


0 06022024 time taken:  3.618
1 07022024 time taken:  5.22
2 08022024 time taken:  3.628
3 09022024 time taken:  1.023
4 10022024 time taken:  5.446
5 11022024 time taken:  5.239
Error tokenizing data. C error: Expected 17 fields in line 1701, saw 18

6 12022024 time taken:  10.955
7 13022024 time taken:  3.514
8 14022024 time taken:  3.431
9 15022024 time taken:  3.501
10 16022024 time taken:  5.206
11 17022024 time taken:  3.488
12 18022024 time taken:  3.538
13 19022024 time taken:  4.215
14 20022024 time taken:  4.235
15 21022024 time taken:  5.102
16 22022024 time taken:  4.275
17 23022024 time taken:  5.171
18 24022024 time taken:  1.448
19 25022024 time taken:  3.789
20 26022024 time taken:  3.587
21 27022024 time taken:  1.122
22 28022024 time taken:  4.963
23 29022024 time taken:  6.924
24 01032024 time taken:  3.628
25 02032024 time taken:  4.002
26 03032024 time taken:  3.66
27 04032024 time taken:  6.816
28 05032024 time taken:  4.223
29 06032024 time taken:  3.652
30 07032

In [3]:
df2=df.copy()
# df2=df2.reset_index()
df2=df2.dropna(subset=['MENU_HINT'])
df2=df2[df2['MENU_HINT'].str.contains('AUS')]
df2['track']=df2['MENU_HINT'].str.split('(').str[0]
df2['EVENT_DT']=df2['EVENT_DT'].astype(str).str.split(' ').str[0]
df2['date']=pd.to_datetime(df2['EVENT_DT'])
df2['name']=df2['SELECTION_NAME'].astype(str).str[3:].str.lower()
df2['track']=df2['track'].str.replace('AUS / ','').str.lower().str.strip()
df2=df2[['track','MENU_HINT','date','name','BSP','MORNINGWAP','PPTRADEDVOL']]
df2


/var/folders/_5/kty37fps3g723664y0dwx0_80000gn/T/ipykernel_13118/1064716288.py:7: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df2['date']=pd.to_datetime(df2['EVENT_DT'])


,track,MENU_HINT,date,name,BSP,MORNINGWAP,PPTRADEDVOL
0,warrnambool,Warrnambool (AUS) 5th Feb,2024-05-02,soho simon says,2.906438,2.3950,6630.51
1,nowra,Nowra (AUS) 5th Feb,2024-05-02,shades of rose,5.543494,3.5252,3047.04
2,maitland,Maitland (AUS) 5th Feb,2024-05-02,hail the king,70.000000,45.9208,320.54
8,angle park,Angle Park (AUS) 5th Feb,2024-05-02,kellys missile,20.481794,14.0907,894.21
9,goulburn,Goulburn (AUS) 5th Feb,2024-05-02,stands tall,4.595880,1.8822,1553.65
...,...,...,...,...,...,...,...
2207,bendigo,Bendigo (AUS) 8th Mar,2024-08-03,wheres walter,5.170479,2.8487,1698.64
2208,bendigo,Bendigo (AUS) 8th Mar,2024-08-03,silky sauce,1.473865,1.2775,30443.47
2211,the gardens,The Gardens (AUS) 8th Mar,2024-08-03,ted robbo,2.401818,2.0034,5149.88
2214,wagga,Wagga (AUS) 8th Mar,2024-08-03,cawbourne mocha,135.485202,45.7284,105.60


In [4]:

dic={
    'shep':'shepparton',
    'mait':'maitland',
    'ball':'ballarat',
    'sprk':'sandown park',
    'goul':'goulburn',
    'hoba':'hobart',
    'dapt':'dapto',
    
    'anpk':'angle park',
    'warr':'warrnambool',
    'wgul':'warragul',
    'aprk':'albion park',
    'nort':'northam',
    'trar':'traralgon',
    'heal':'healesville',
    'rich':'richmond',
    
    'ipsw':'ipswich',
    'geel':'geelong',
    'casi':'casino',
    'gard':'the gardens',
    'cann':'cannington',
    'mgam':'mount gambier',
    'dubb':'dubbo',
    
    'bend':'bendigo',
    'bull':'bulli',
    'hshm':'horsham',
    'wprk':'wentworth park',
    'mead':'the meadows',
    'sale':'sale',
    'darw':'darwin',
    
    'gawl':'gawler',
    'graf':'grafton',
    'nowr':'nowra',
    'bath':'bathurst',
    'laun':'launceston',
    'town':'townsville',
    'devn':'devonport',
    
    'lism':'lismore',
    'rock':'rockhampton',
    'cran':'cranbourne',
    'stra':'strathalbyn',
    'wagg':'wagga',
    'crns':'cairns',
    'mand':'mandurah',
    
    'temo':'temora',
    'gosf':'gosford',
    'capa':'capalaba',
    'mbdg':'murray bridge',
    
    'anprk':'angle park',
    'ispw':'ipswich',
    'bund':'bundaberg',
    'gunn':"gunnedah",
    'tare':'taree'
    
    
}

correction=True
if(correction):
    for tr in dic.keys():
        df2['track']=np.where(df2['track']==tr,dic[tr],df2['track'])
print(df2['track'].unique())


['warrnambool' 'nowra' 'maitland' 'angle park' 'goulburn' 'traralgon'
 'cannington' 'bundaberg' 'geelong' 'shepparton' 'richmond' 'launceston'
 'albion park' 'murray bridge' 'horsham' 'mandurah' 'warragul' 'gosford'
 'ipswich' 'casino' 'bulli' 'townsville' 'healesville' 'rockhampton'
 'the meadows' 'capalaba' 'temora' 'muswellbrook' 'gawler' 'darwin'
 'hobart' 'bendigo' 'gunnedah' 'mount gambier' 'sandown park' 'dapto'
 'the gardens' 'wentworth park' 'wagga' 'dubbo' 'sale' 'grafton'
 'ballarat']


In [5]:
files = os.listdir('bsp')
files = [x for x in files if '_' in x]
files = [x for x in files if '.D' not in x]
#files = [int(x.split('_')[1].strip('.csv')) for x in files]

try:
    nex = max(files)+1
except:
    nex=0
fname = 'bsp/bsp2025.csv'
df2.to_csv(fname)